## NFI Wood Production, Planting and Restocking

This workflow deals with analysus of National Forest Inventory's figures on Wood Production, Planting and Restocking. The goal is to procude supplimentary figured regarding the real extent of canopy cover in the UK, it's dynamics as well as the dynamics of it's reduction.

Secondary motivation for these figures inclusion was the attempt to explain the loss of canopy cover that ocurred at an average rate of `35,000 ha` annualy from 2012 to 2022.

### NFI Methodology Note

It's important to realise that NFI calculates a year as "year ending 31 March".

### Yield per hectare

Unfortunately, there is no direct report of the yield per hectare in the NFI data. However, we can use their statistics, analytics and other reports as well as industry standards to ballpark the figure.

There is a comprehensive report on Forest Yield calculation and tables available from [Forest Research](https://www.forestresearch.gov.uk/tools-and-resources/fthr/forest-yield/). It suggests that, whilst our calculation is generally correct it might not be representative of typical, conifer-dominated forestry estates. 

Generally, based on reports of `8-12 m3/ha` yield with a mean maturity of `40 years` which produces an average of `320 - 480 m3/ha` or or `485 green tonnes / ha`.

#### Inferring hectares removed from wood production
There is an alternative way to try and calculate the yield per hectare based on NFI reporting figures, but the process is highly flawed. The calculations return a yield of~ `250 gt/ha`, meaning each heactare of forestry cut down at mean 40 years would prodice 250 green tonnes of timber. This is a very low figure and is likely to be a result of standing volume with area of woodland with trees not being a good indicator of the actual yield.

#### Unit Conversion

![Unit Conversion](../assets/unit_conversion.png)

The NFI's Unit Conversion table can be used to convert between Green Tonnes, Standing Volume, Overbark and Underbark volumes. Source

In [ ]:
import pandas as pd

hectare_yield = 480

def process_nfi_wood_production():
    # Define the sheet names and countries
    sheets_countries = [('Table_1', 'England'), ('Table_2', 'Wales'), ('Table_3', 'Scotland'), ('Table_5', 'Northern Ireland')]

    # Define the column name mappings
    column_mappings = {
        'Table_1': {'FE Softwood': 'Public Softwood', 'FE Hardwood': 'Public Hardwood'},
        'Table_2': {'NRW Softwood': 'Public Softwood', 'NRW Hardwood': 'Public Hardwood'},
        'Table_3': {'FLS Softwood': 'Public Softwood', 'FLS Hardwood': 'Public Hardwood'},
        'Table_5': {'FS Softwood': 'Public Softwood', 'FS Hardwood': 'Public Hardwood'}
    }

    # Initialize an empty list to store the dataframes
    dfs = []

    # Process each table
    for sheet_name, country in sheets_countries:
        # Load data and select desired range
        df = pd.read_excel('../data/sheets/uk_nfi_wood_production.ods', sheet_name=sheet_name, engine='odf').iloc[3:52, 0:7]

        # Set column names and drop first row
        df.columns = df.iloc[0]
        df = df.iloc[1:]

        # Drop 'Total' columns and rename others
        df = df.drop(columns=['Total Softwood', 'Total Hardwood']).rename(columns={
            'Private Sector Softwood': 'Private Softwood',
            'Private Sector Hardwood': 'Private Hardwood',
            **column_mappings[sheet_name]
        })

        # Melt DataFrame into long format and split 'Type and Ownership' column
        df = df.melt(id_vars='Year', var_name='Type and Ownership', value_name='Volume')
        df[['Ownership', 'Type']] = df['Type and Ownership'].str.split(' ', expand=True)
        df = df.drop(columns='Type and Ownership')

        # Reorder columns and add 'Country' column
        df = df[['Year', 'Type', 'Ownership', 'Volume']].assign(Country=country)
        df = df[['Year', 'Country', 'Type', 'Ownership', 'Volume']]

        # Bringing to green tonnes
        df['Volume'] = df['Volume']*1000

        # Append the dataframe to the list
        dfs.append(df)

    # Concatenate all dataframes
    df_all = pd.concat(dfs).reset_index(drop=True)

    return df_all

def process_nfi_wood_restocking():
    # Define the sheet names and countries
    sheets_countries = [('Table_C1', 'England'), ('Table_C2', 'England'), ('Table_C3', 'Wales'), ('Table_C4', 'Wales'), ('Table_C5', 'Scotland'), ('Table_C6', 'Scotland'), ('Table_C7', 'Northern Ireland'), ('Table_C8', 'Northern Ireland')]

    # Define the column name mappings
    column_mappings = {
        'Table_C1': {'FE conifers (thousand ha)': 'Public Conifers', 'FE broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C2': {'FE conifers (thousand ha)': 'Public Conifers', 'FE broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C3': {'NRW conifers (thousand ha)': 'Public Conifers', 'NRW broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C4': {'NRW conifers (thousand ha)': 'Public Conifers', 'NRW broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C5': {'FLS conifers (thousand ha)': 'Public Conifers', 'FLS broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C6': {'FLS conifers (thousand ha)': 'Public Conifers', 'FLS broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C7': {'FS conifers (thousand ha)': 'Public Conifers', 'FS broadleaves (thousand ha)': 'Public Broadleaves'},
        'Table_C8': {'FS conifers (thousand ha)': 'Public Conifers', 'FS broadleaves (thousand ha)': 'Public Broadleaves'}
    }

    # Initialize an empty list to store the dataframes
    dfs = []

    # Process each table
    for sheet_name, country in sheets_countries:
        # Load data and select desired range
        df = pd.read_excel('../data/sheets/uk_nfi_planting_restocking.ods', sheet_name=sheet_name, engine='odf').iloc[3:999, 0:6]

        # Set column names and drop first row
        df.columns = df.iloc[0]
        df = df.iloc[1:]

        # Drop 'Total' column and rename others
        df = df.drop(columns=['Total (thousand ha)']).rename(columns={
            'Private sector conifers (thousand ha)': 'Private Conifers',
            'Private sector broadleaves (thousand ha)': 'Private Broadleaves',
            **column_mappings[sheet_name]
        })

        # Melt DataFrame into long format and split 'Type and Ownership' column
        df = df.melt(id_vars='Year ending 31 March', var_name='Type and Ownership', value_name='Area')
        df[['Ownership', 'Type']] = df['Type and Ownership'].str.split(' ', n=1, expand=True)
        df = df.drop(columns='Type and Ownership')

        # Reorder columns and add 'Country' and 'Activity' columns
        activity = 'Planting' if sheet_name in ['Table_C1', 'Table_C3', 'Table_C5', 'Table_C7'] else 'Restocking'
        df = df.rename(columns={'Year ending 31 March': 'Year'}).assign(Country=country, Activity=activity)
        df = df[['Year', 'Country', 'Activity', 'Type', 'Ownership', 'Area']]

        # Replace '[low]' with 0 and remove rows with '[x]', bring to hectares
        df['Area'] = df['Area'].replace('[low]', 0.005)*1000
        df = df[df['Area'] != '[x]']

        # Filter out rows before 1976
        df = df[df['Year'] >= 1976]

        # Append the dataframe to the list
        dfs.append(df)

    # Concatenate all dataframes and reset index
    df_all_restocking = pd.concat(dfs).reset_index(drop=True)

    return df_all_restocking

In [ ]:
nfi_wood_production = process_nfi_wood_production()
nfi_planting_restocking = process_nfi_wood_restocking()

In [ ]:
nfi_woodland_reduction = nfi_wood_production.copy()
nfi_woodland_reduction['Area'] = nfi_woodland_reduction['Volume'] / hectare_yield
nfi_woodland_reduction = nfi_woodland_reduction.drop(columns='Volume')

In [ ]:
nfi_planting_restocking

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def interactive_plot(df):
    # Create multi-select widgets for the filters
    country_widget = widgets.SelectMultiple(options=['Combined'] + list(df['Country'].unique()), value=['Combined'], description='Country:')
    type_widget = widgets.SelectMultiple(options=['Combined'] + list(df['Type'].unique()), value=['Combined'], description='Type:')
    ownership_widget = widgets.SelectMultiple(options=['Combined'] + list(df['Ownership'].unique()), value=['Combined'], description='Ownership:')

    # Function to update the plot based on selected filters
    def update_plot(country, type_, ownership):
        # Filter the data
        df_filtered = df.copy()
        if 'Combined' not in country:
            df_filtered = df_filtered[df_filtered['Country'].isin(country)]
        if 'Combined' not in type_:
            df_filtered = df_filtered[df_filtered['Type'].isin(type_)]
        if 'Combined' not in ownership:
            df_filtered = df_filtered[df_filtered['Ownership'].isin(ownership)]

        # Create the plot
        plt.figure(figsize=(10, 6))
        group_by_columns = []
        if 'Combined' in country:
            group_by_columns.append('Country')
        if 'Combined' in type_:
            group_by_columns.append('Type')
        if 'Combined' in ownership:
            group_by_columns.append('Ownership')
        df_grouped = df_filtered.groupby(group_by_columns + ['Year'])['Area'].sum().reset_index()
        for key, grp in df_grouped.groupby(group_by_columns):
            label = 'Combined' if 'Combined' in key else key
            plt.plot(grp['Year'], grp['Area'], label=label)

        # Set the title and labels
        plt.title('Annual area of woodland removed, approx.')
        plt.xlabel('Year')
        plt.ylabel('Area')
        plt.legend()

        # Show the plot
        plt.show()

    # Use interactive widget
    widgets.interact(update_plot, country=country_widget, type_=type_widget, ownership=ownership_widget)

In [ ]:
interactive_plot(nfi_woodland_reduction)

In [ ]:
interactive_plot(nfi_planting_restocking)